# COVID - 19

Questions:



Datasource: Google Cloud - Public Datasets, Covid19_usafacts

In [1]:
import pandas as pd

In [2]:
#create dataframe 
cases = pd.read_csv(r'C:\Users\Chunna\Documents\Jupterlab\Covid\confirmed_cases.csv')
deaths = pd.read_csv(r'C:\Users\Chunna\Documents\Jupterlab\Covid\deaths.csv')


In [18]:
#manipulate data: remove columns & unpivot
cases_drop = cases.drop(columns=['county_fips_code','state_fips_code'])
cases_unpivot = cases_drop.melt(id_vars=['county_name','state'], var_name='date', value_name='rolling_confirmed cases')
cases_unpivot['date'] = cases_unpivot.date.str[1:].replace('_', '-',regex=True)
#deaths_drop = deaths.drop(columns=['county_fips_code','state_fips_code'])
deaths_unpivot = deaths_drop.melt(id_vars=['county_name','state'], var_name='date', value_name='rolling_deaths')
deaths_unpivot['date'] = deaths_unpivot.date.str[1:].replace('_', '-',regex=True)


#remove rolling deaths
deaths_unpivot['deaths_calc'] = deaths_unpivot.groupby(['state'])['rolling_deaths'].shift(1)
deaths_unpivot.loc[deaths_unpivot['deaths_calc'] == deaths_unpivot['rolling_deaths'], 'deaths'] = True



county = deaths_unpivot['county_name'] == 'Hopkins County '
state = deaths_unpivot['state'] == 'KY'
date = deaths_unpivot['date'] == '2021-09-21'
deaths_unpivot[county & state].sort_values(['date'], ascending=True)
#each date has rolling total


,county_name,state,date,rolling_deaths,deaths_calc,deaths
88,Hopkins County,KY,2020-01-22,0,NaN,NaN
1159,Hopkins County,KY,2020-01-23,0,0.0,True
2230,Hopkins County,KY,2020-01-24,0,0.0,True
3301,Hopkins County,KY,2020-01-25,0,0.0,True
4372,Hopkins County,KY,2020-01-26,0,0.0,True
...,...,...,...,...,...,...
646972,Hopkins County,KY,2021-09-17,177,81.0,NaN
648043,Hopkins County,KY,2021-09-18,177,81.0,NaN
649114,Hopkins County,KY,2021-09-19,177,81.0,NaN
650185,Hopkins County,KY,2021-09-20,177,81.0,NaN


In [4]:
#merge dataframes
df = pd.merge(df_cases,df_deaths)
##df = df.groupby(['state','date']).agg({'confirmed cases':'sum','deaths':'sum'}).reset_index()
df

NameError: name 'df_cases' is not defined

## Which top 5 states has the most covid cases?

In [ ]:
cases_unpivot.groupby('state').sum().sort_values('cases', ascending=False).head(5).round({'cases': 0}).astype(int).reset_index()

## Which state had the highest covid cases?

In [ ]:
cases_unpivot.max()

## Which state had the most covid cases?

In [ ]:
cases_unpivot.groupby('state').sum()